In [46]:
import numpy as np
import geopandas as gpd

In [47]:
str_input_gpkg = r'E:\sample_2d_output\streams_mannings.gpkg'
str_limiting_discharge_csv = r'E:\ras2fim-2d\input_gis\LimitingDischarge.csv'

In [48]:
# *************
# For now (2024.05.07 - the upper flow is 30% of the limiting discharge)
flt_q_ratio = 0.30
# *************

In [49]:
def fn_calculate_travel_time(row):
    
    # *************
    #For a low flow travel time, this is the Rh that is being assumed (need a conservative estimate)
    flt_assumed_hydraulic_radius = 1.1
    # *************
    
    flt_mannings = row['manning']
    flt_length_m = row['lengthkm'] * 1000
    flt_slope_m_per_m = row['So']
    flt_time_sec = (flt_length_m * flt_mannings) / ((flt_assumed_hydraulic_radius ** 0.667) * (flt_slope_m_per_m ** 0.5))
    flt_time_hr = round(flt_time_sec / 3600, 2)
    return flt_time_hr

In [50]:
# Load the GeoDataFrames
gdf_streams = gpd.read_file(str_input_gpkg)

# Load the CSV file into a NumPy array
data = np.genfromtxt(str_limiting_discharge_csv, delimiter=',')
data = data[data[:,0].argsort()]

gdf_streams['da_sq_mile'] = gdf_streams['tot_drainage_areasqkm'] * 0.386102

# Perform linear interpolation for each row in df and round the result to the nearest integer
gdf_streams['q_limiting'] = gdf_streams['da_sq_mile'].apply(lambda x: round(np.interp(x, data[:,0], data[:,1])))

# Upper flow limit is ratio of limiting discharge == flt_q_ratio
gdf_streams['q_upper_limit'] = round(gdf_streams['q_limiting'] * flt_q_ratio,0)

# Assuming gdf_streams is your GeoDataFrame
gdf_streams['travel_time_hr'] = gdf_streams.apply(fn_calculate_travel_time, axis=1)

In [51]:
gdf_streams

,id,toid,mainstem,order,hydroseq,lengthkm,areasqkm,tot_drainage_areasqkm,has_divide,divide_id,...,So,ChSlp,within_2darea,network_group,manning,geometry,da_sq_mile,q_limiting,q_upper_limit,travel_time_hr
0,wb-2410251,nex-2410252,1884413.0,3.0,23858,3.611474,6.762601,105.379651,True,cat-2410251,...,0.004384,0.393827,True,0,0.061,"MULTILINESTRING ((-201974.170 865603.245, -201...",40.687294,159844,47953.0,0.87
1,wb-2410255,nex-2410256,1884413.0,3.0,23880,4.242299,12.443399,298.642500,True,cat-2410255,...,0.002474,0.335564,True,0,0.060,"MULTILINESTRING ((-191536.885 862432.472, -191...",115.306467,300272,90082.0,1.33
2,wb-2410258,nex-2410259,1884413.0,3.0,23891,3.307006,4.692601,402.482251,True,cat-2410258,...,0.002290,0.321687,True,0,0.060,"MULTILINESTRING ((-183575.970 855531.666, -183...",155.399202,359858,107957.0,1.08
3,wb-2410259,nex-2410260,1884413.0,3.0,23902,5.117748,9.213300,513.826651,True,cat-2410259,...,0.002406,0.310669,True,0,0.060,"MULTILINESTRING ((-183012.311 852973.757, -182...",198.389498,416970,125091.0,1.63
4,wb-2410260,nex-2410261,1884413.0,3.0,23906,4.738277,11.139302,543.650852,True,cat-2410260,...,0.002356,0.306763,True,0,0.060,"MULTILINESTRING ((-180670.016 849786.164, -180...",209.904681,431306,129392.0,1.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,wb-2410263,nex-2410264,1884413.0,3.0,23912,8.439995,34.848450,638.368652,True,cat-2410263,...,0.000820,0.298813,True,0,0.059,"MULTILINESTRING ((-171561.658 847291.965, -171...",246.475413,474553,142366.0,4.53
68,wb-2410264,nex-2410265,1884413.0,3.0,23913,1.725184,4.174200,642.542852,True,cat-2410264,...,0.004500,0.298149,True,0,0.057,"MULTILINESTRING ((-165132.694 844992.882, -165...",248.087080,476391,142917.0,0.38
69,wb-2410523,nex-2410487,1884821.0,1.0,23917,3.787704,3.903300,3.903300,True,cat-2410523,...,0.009520,0.641737,False,1,0.070,"MULTILINESTRING ((-204837.470 854732.676, -204...",1.507072,13937,4181.0,0.71
70,wb-2410509,nex-2410491,1884794.0,1.0,23936,3.767248,3.233250,3.233250,True,cat-2410509,...,0.010070,0.660385,False,2,0.100,"MULTILINESTRING ((-187569.358 849434.073, -187...",1.248364,11819,3546.0,0.98
